In [659]:
from Bio import SeqIO
import pandas as pd
import time as t

In [687]:
class Node(object):
    def __init__(self, data, val, res_ind, sp, var):
        self.variant = var
        self.species = sp
        self.data = data
        self.value = val
        self.res_ind = res_ind
        self.parent = None
        self.height = 0
        self.left_child = None
        self.right_child = None


class AVLTree(object):

    def __init__(self):
        self.root = None

    def insert(self, data, val, res_ind, sp, var):
        if self.root is None:
            self.root = Node(data, val, res_ind, sp, var)
        else:
            new_node = self._insert(self.root, data, val, res_ind, sp, var)
            self._walk_up(new_node)

    def _insert(self, node, data, val, res_ind, sp, var):
        if data <= node.data:
            if not node.left_child:
                node.left_child = Node(data, val, res_ind, sp, var)
                node.left_child.parent = node
                return node.left_child
            return self._insert(node.left_child, data, val, res_ind, sp, var)
        else:
            if not node.right_child:
                node.right_child = Node(data, val, res_ind, sp, var)
                node.right_child.parent = node
                return node.right_child
            return self._insert(node.right_child, data, val, res_ind, sp, var)

    def remove(self, data):
        if not self.root:
            raise ValueError('Tree is empty.')
        self._delete_value(data)

    def _delete_value(self, data):
        node = self.find(data, self.root)
        if node is False:
            raise ValueError('No node with value {}'.format(data))
        parent_of_deleted_node = self._delete_node(node)
        # if parent is None we know that we just deleted the root node,
        # but if root node had a child, that child is now the root node!
        if not parent_of_deleted_node and self.root:
            parent_of_deleted_node = self.root
        self._walk_up(parent_of_deleted_node)

    def find(self, value, node):
        if node is None:
            return False
        if node.data > value:
            return self.find(value, node.left_child)
        elif node.data < value:
            return self.find(value, node.right_child)
        return node

    def _delete_node(self, node):
        parent_node = node.parent
        num_child = self._num_children(node)

        if num_child == 0:
            # If there is no parent then 'node' is the root node.
            # 'node' has no children, so set root to point to None.
            if not parent_node:
                self.root = None
            elif parent_node.left_child == node:
                parent_node.left_child = None
            else:
                parent_node.right_child = None
            return parent_node

        elif num_child == 1:
            if node.left_child:
                child = node.left_child
            else:
                child = node.right_child

            if not parent_node:
                self.root = child
            elif parent_node.left_child == node:
                parent_node.left_child = child
            else:
                parent_node.right_child = child
            child.parent = parent_node
            return parent_node

        else:
            successor = self._max_node(node.left_child)
            node.data = successor.data
            self._delete_node(successor)

    @staticmethod
    def _num_children(node):
        num_children = 0
        if node.left_child:
            num_children += 1
        if node.right_child:
            num_children += 1
        return num_children

    def _walk_up(self, node):
        if not node:
            return
        else:
            self._check_node(node)
            return self._walk_up(node.parent)

    def _check_node(self, node):
        left_height = -1
        right_height = -1
        if node.left_child:
            left_height = node.left_child.height
        if node.right_child:
            right_height = node.right_child.height
        if abs(left_height - right_height) > 1:
            if left_height < right_height:
                self.left_rotate(node, node.right_child)
            else:
                self.right_rotate(node, node.left_child)
        else:
            node.height = max(left_height, right_height) + 1

    def left_rotate(self, node, child_node):
        if child_node.left_child:
            node.right_child = child_node.left_child
            node.right_child.parent = node
        else:
            node.right_child = None

        if node != self.root:
            child_node.parent = node.parent
            if node.parent.right_child == node:
                node.parent.right_child = child_node
            else:
                node.parent.left_child = child_node
        else:
            child_node.parent = None
            # because we are not replacing the parent node('node') with the
            # child node('child_node'), self.root is still pointing at the parent node.
            self.root = child_node

        child_node.left_child = node
        node.parent = child_node

        node.height -= 1

    def right_rotate(self, node, child_node):
        if child_node.right_child:
            node.left_child = child_node.right_child
            node.left_child.parent = node
        else:
            node.left_child = None

        if node != self.root:
            child_node.parent = node.parent
            if node.parent.right_child == node:
                node.parent.right_child = child_node
            else:
                node.parent.left_child = child_node
            # node.parent.left_child = child_node
        else:
            child_node.parent = None
            self.root = child_node

        child_node.right_child = node
        node.parent = child_node

        node.height -= 1

    def _min_node(self, node):
        if node.left_child:
            return self._min_node(node.left_child)
        return node

    def _max_node(self, node):
        if node.right_child:
            return self._max_node(node.right_child)
        return node

    def traverse(self, method='in'):
        if not self.root:
            return iter(())
        if method == 'in':
            return self._traverse_inorder(self.root)
        elif method == 'pre':
            return self._traverse_preorder(self.root)
        elif method == 'post':
            return self._traverse_postorder(self.root)
        else:
            raise ValueError('method must be either "in", "pre" or "post".')

    # left subtree -> root -> right subtree
    def _traverse_inorder(self, node):
        if node.left_child:
            yield from self._traverse_inorder(node.left_child)
        yield node
        if node.right_child:
            yield from self._traverse_inorder(node.right_child)

    # root -> left subtree -> right subtree
    def _traverse_preorder(self, node):
        yield node
        if node.left_child:
            yield from self._traverse_inorder(node.left_child)
        if node.right_child:
            yield from self._traverse_inorder(node.right_child)

    # left subtree -> right subtree -> root
    def _traverse_postorder(self, node):
        if node.left_child:
            yield from self._traverse_inorder(node.left_child)
        if node.right_child:
            yield from self._traverse_inorder(node.right_child)
        yield node




tree = AVLTree()

In [688]:
def get_species(descr):
    species = descr.split('|', 2)[1]
    return species

print(get_species('>H1.0|Bos|115496898 Bos|115496898|H1.0 Bos_H1.0_115496898'))

Bos


In [689]:
def get_variant(descr):
    variant = descr.split('|', 1)[0].replace('>', '')
    return variant

print(get_variant('>H1.0|Bos|115496898 Bos|115496898|H1.0 Bos_H1.0_115496898'))

H1.0


In [690]:
# open files

path = '../histones/aligned_variants'
h1 = open(path + '/H1.fasta')
h2a = open(path + '/H2A.fasta')
h2b = open(path + '/H2B.fasta')
h3 = open(path + '/H3.fasta')
h4 = open(path + '/H4.fasta')


# add histone alignments from files into avl_tree
start = t.time()

files = [h1, h2a, h2b, h3, h4]
types = ['H1', 'H2A', 'H2B', 'H3', 'H4']
rests = {'H1': 0, 'H2A': 1, 'H2B': 2, 'H3':  3, 'H4': 4}
id_counter = {'H1': 0, 'H2A': 1, 'H2B': 2, 'H3':  3, 'H4': 4}
his_counter = 0
i = -1

for file in files:
    i += 1
    cur_his_type = types[i]
    for record in SeqIO.parse(file, 'fasta'):
        res_ind = [i for i in range(len(record.seq)) if record.seq[i] != '-']
        species = get_species(record.description)
        variant = get_variant(record.description)
        his_id = id_counter[cur_his_type]
        id_counter[cur_his_type] += 6
        his_counter += 1
        tree.insert(his_id, record, res_ind, species, variant)
        
                
print(t.time() - start)
print(his_counter)
print(id_counter)

0.11520719528198242
344
{'H1': 342, 'H2A': 913, 'H2B': 350, 'H3': 381, 'H4': 88}


In [691]:
# read contact table and delit letters p and d from histone names

conts = pd.read_csv('histone_contacts.csv')
conts = conts[(conts['A_entity'] != conts['B_entity'])]

conts['A_entity'] = [his[:-1] for his in conts['A_entity']]
conts['B_entity'] = [his[:-1] for his in conts['B_entity']]
conts

,Unnamed: 0,A_segid,A_resid,A_resname,B_segid,B_resid,B_resname,num_int,A_entity,B_entity
202,202,A,44,G,B,44,K,2,H3,H4
227,227,A,47,A,B,39,R,5,H3,H4
228,228,A,47,A,B,44,K,1,H3,H4
238,238,A,48,L,B,44,K,2,H3,H4
239,239,A,48,L,G,115,L,2,H3,H2A
...,...,...,...,...,...,...,...,...,...,...
8294,8294,H,118,Y,G,20,R,2,H2B,H2A
8295,8295,H,118,Y,G,49,V,2,H2B,H2A
8317,8317,H,121,A,G,20,R,1,H2B,H2A
8324,8324,H,122,K,G,6,Q,2,H2B,H2A


In [693]:
start = t.time()

res_list = [[] for k in range(len(conts))]
i = -1

for j, row in conts.iterrows():
    a_his, b_his = row[8], row[9]
    a_resi, b_resi = row[2] - 1, row[5] - 1
    a_resname, b_resname = row[3], row[6]
    a_start, b_start = rests[a_his], rests[b_his]
    a_stop, b_stop = id_counter[a_his], id_counter[b_his]
    i += 1

    while a_start < a_stop:
        a_node = tree.find(a_start, tree.root)
        a_start += 6
        a_seq = a_node.value.seq
        try:
            a_new_resname = a_seq[a_node.res_ind[a_resi]]
        except IndexError:
            continue
        else:
            if a_new_resname != a_resname:
                
                while b_start < b_stop:
                    b_node = tree.find(b_start, tree.root)
                    b_start += 6
                    b_seq = b_node.value.seq
                    try:
                        b_new_resname = b_seq[b_node.res_ind[b_resi]]
                    except IndexError:
                        continue
                    else:
                        if b_new_resname != b_resname and a_node.species == b_node.species:
                            res_list[i].append((species, a_his, b_his,
                                                'old', a_resid, a_resname, b_resid, b_resname,
                                                'new', a_new_resname, b_new_resname,
                                                a_node.variant, b_node.variant))
        
print(t.time() - start)

1.2945806980133057


In [695]:
print(res_list)
print(len(res_list))

[[('Zea', 'H3', 'H4', 'old', 44, 'G', 44, 'K', 'new', 'P', 'V', 'H3.3', 'canonical_H4')], [('Zea', 'H3', 'H4', 'old', 44, 'A', 44, 'R', 'new', 'V', 'A', 'H3.3', 'canonical_H4')], [('Zea', 'H3', 'H4', 'old', 44, 'A', 44, 'K', 'new', 'V', 'V', 'H3.3', 'canonical_H4')], [('Zea', 'H3', 'H4', 'old', 44, 'L', 44, 'K', 'new', 'A', 'V', 'H3.3', 'canonical_H4')], [('Zea', 'H3', 'H2A', 'old', 44, 'L', 44, 'L', 'new', 'A', 'G', 'H3.3', 'H2A.W'), ('Zea', 'H3', 'H2A', 'old', 44, 'L', 44, 'L', 'new', 'A', 'S', 'H3.3', 'H2A.W'), ('Zea', 'H3', 'H2A', 'old', 44, 'L', 44, 'L', 'new', 'A', 'V', 'H3.3', 'H2A.X'), ('Zea', 'H3', 'H2A', 'old', 44, 'L', 44, 'L', 'new', 'A', 'V', 'H3.3', 'H2A.X'), ('Zea', 'H3', 'H2A', 'old', 44, 'L', 44, 'L', 'new', 'A', 'A', 'H3.3', 'H2A.Z'), ('Zea', 'H3', 'H2A', 'old', 44, 'L', 44, 'L', 'new', 'A', 'T', 'H3.3', 'H2A.Z'), ('Zea', 'H3', 'H2A', 'old', 44, 'L', 44, 'L', 'new', 'A', 'T', 'H3.3', 'H2A.Z'), ('Zea', 'H3', 'H2A', 'old', 44, 'L', 44, 'L', 'new', 'A', 'P', 'H3.3', 'H2A